# Fiddler examples have moved! [Deprecation Notice]

Dear user thank you for using fiddler product, we appreciate your time! We have moved the examples to a new github repo located at the following link


***
# [New fiddler-examples repo](https://github.com/fiddler-labs/fiddler-examples)
***

# Fiddler Customer Churn

## Event Publishing

In [3]:
import numpy as np
import pandas as pd
import fiddler as fdl

print(f"Running client version {fdl.__version__}")

Running client version 1.0.2


## 1. Connect to Fiddler

Before you can register your model with Fiddler, you'll need to connect using our API client.


---


**We need a few pieces of information to get started.**
1. The URL you're using to connect to Fiddler

In [4]:
URL = ''

2. Your organization ID
3. Your authorization token

Both of these can be found by clicking the URL you entered and navigating to the **Settings** page.

In [5]:
ORG_ID = ''
AUTH_TOKEN = ''

In [6]:
client = fdl.FiddlerApi(
    url=URL,
    org_id=ORG_ID,
    auth_token=AUTH_TOKEN
)

fiddler.connection INFO client_version = 1.0.2 > server_version = 1.0.1
fiddler.connection WARNING OUTDATED SERVER: The URL you connected to is running version 1.0.1, consider getting it upgraded to match your client 1.0.2.


### Publish production events


Fiddler will **monitor this data and compare it to your baseline to generate powerful insights into how your model is behaving**.


---

In [26]:
PROJECT_ID = ''
MODEL_ID = ''
DATASET_ID = ''

In [27]:
PATH_TO_EVENTS_CSV = 'https://raw.githubusercontent.com/fiddler-labs/fiddler-samples/new-quickstart/content_root/tutorial/quickstart/churn_events.csv'

production_df = pd.read_csv(PATH_TO_EVENTS_CSV)
production_df

,creditscore,geography,gender,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,churn,predicted_churn,decision,timestamp
0,559,California,Male,52,2,0.00,1,1,0,129013.59,no,0.007448,low_risk,1628610458681
1,482,California,Male,55,5,97318.25,1,0,1,78416.14,yes,0.804852,high_risk,1628612877881
2,651,Florida,Female,46,4,89743.05,1,1,0,156425.57,no,0.012754,low_risk,1628615297081
3,611,Hawaii,Male,38,7,0.00,1,1,1,63202.00,yes,0.882252,high_risk,1628617716281
4,696,California,Female,33,4,0.00,2,1,1,73371.65,yes,0.999736,high_risk,1628620135481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,781,Hawaii,Female,48,0,57098.96,6,1,0,85644.06,no,0.032330,low_risk,1629203162681
246,797,Hawaii,Female,55,10,0.00,9,1,1,49418.87,no,0.020316,low_risk,1629205581881
247,554,Hawaii,Male,31,1,0.00,7,0,1,192660.55,yes,0.269628,low_risk,1629208001081
248,701,Hawaii,Nonbinary,37,1,0.00,7,1,0,163457.55,yes,0.769625,high_risk,1629210420281


In [30]:
production_df['timestamp'] = pd.to_datetime(production_df['timestamp'], unit='ms')

In [32]:
production_df['timestamp'] = pd.to_datetime(production_df['timestamp'], format="%Y-%m-%d %H:%M:%S:%f")

In [36]:
production_df['timestamp'] = production_df['timestamp'].astype(str)

In [37]:
production_df.head()

,creditscore,geography,gender,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,churn,predicted_churn,decision,timestamp
0,559,California,Male,52,2,0.00,1,1,0,129013.59,no,0.007448,low_risk,2021-08-10 15:47:38.681
1,482,California,Male,55,5,97318.25,1,0,1,78416.14,yes,0.804852,high_risk,2021-08-10 16:27:57.881
2,651,Florida,Female,46,4,89743.05,1,1,0,156425.57,no,0.012754,low_risk,2021-08-10 17:08:17.081
3,611,Hawaii,Male,38,7,0.00,1,1,1,63202.00,yes,0.882252,high_risk,2021-08-10 17:48:36.281
4,696,California,Female,33,4,0.00,2,1,1,73371.65,yes,0.999736,high_risk,2021-08-10 18:28:55.481


In [38]:
production_events = production_df.to_dict(orient='records')
production_events[0]

{'creditscore': 559,
 'geography': 'California',
 'gender': 'Male',
 'age': 52,
 'tenure': 2,
 'balance': 0.0,
 'numofproducts': 1,
 'hascrcard': 1,
 'isactivemember': 0,
 'estimatedsalary': 129013.59,
 'churn': 'no',
 'predicted_churn': 0.0074475368963339,
 'decision': 'low_risk',
 'timestamp': '2021-08-10 15:47:38.681'}

In [39]:
from tqdm import tqdm

for event in tqdm(production_events):

    client.publish_event(
        project_id=PROJECT_ID,
        model_id=MODEL_ID,
        event=event,
        event_timestamp=event['timestamp']
    )

100%|█████████████████████████████████████████| 250/250 [00:29<00:00,  8.50it/s]


### Use the Fiddler Platform

In [11]:
print('/'.join([URL, 'projects', PROJECT_ID, 'models', MODEL_ID, 'monitor']))

https://vishal.staging.fiddler.ai/projects/airflow_churn_example_2/models/churn_classifier/monitor
